This is where I am experimenting with how to call and download the data from datahub's Covid-19 database

The location of the data site is https://datahub.io/core/covid-19#data
The files are in both JSON and CSV file format

US data is located at /core/covid-19/r/us_confirmed.cvs (or .json)
US data is located at /core/covid-19/r/us_deaths.cvs (or .json)

In [1]:
import requests
import bs4
import numpy
import pandas

In [ ]:
## Code to search the datahub website to find and download the latest data.
## This can of course be addressed manually but for practice this code would 
## help me search for the link to download the data

base_url ="https://datahub.io/core/covid-19#data"
res = bs4.BeautifulSoup(request.get(base_url).text,"lxml")
res.select(#Input search criteria here)